<a href="https://colab.research.google.com/github/AkshatPatel15/crime-report/blob/main/joins.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 55 kB/s 
     |████████████████████████████████| 199 kB 35.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=0c506e83e6fb0c9947fed65657a90577e71ce0c3cc3ebf242339dd31fdc7276c
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
# ! wget "https://data.cityofchicago.org/api/views/z8bn-74gv/rows.csv?accessType=DOWNLOAD"

--2022-09-23 18:35:25--  https://data.cityofchicago.org/api/views/z8bn-74gv/rows.csv?accessType=DOWNLOAD
Resolving data.cityofchicago.org (data.cityofchicago.org)... 52.206.68.26, 52.206.140.199, 52.206.140.205
Connecting to data.cityofchicago.org (data.cityofchicago.org)|52.206.68.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘rows.csv?accessType=DOWNLOAD’

rows.csv?accessType     [ <=>                ]   5.57K  --.-KB/s    in 0s      

2022-09-23 18:35:26 (453 MB/s) - ‘rows.csv?accessType=DOWNLOAD’ saved [5699]



In [7]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [8]:
# ! mv police_station.csv drive/MyDrive/google\ colab/police_station.csv

In [9]:
! cp drive/MyDrive/google\ colab/police_station.csv ../content/police_station.csv

In [10]:
# ! ls drive/MyDrive/google\ colab

In [11]:
! cp drive/MyDrive/google\ colab/colab.csv ../content/crime.csv

In [12]:
!ls

crime.csv  drive  police_station.csv  sample_data


In [13]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [14]:
spark

In [15]:
df_police_station = spark.read.csv('police_station.csv',header=True)
df_police_station.show(5)

+------------+-------------+-------------------+-------+-----+-----+--------------------+------------+------------+------------+------------+------------+-----------+------------+--------------------+
|    DISTRICT|DISTRICT NAME|            ADDRESS|   CITY|STATE|  ZIP|             WEBSITE|       PHONE|         FAX|         TTY|X COORDINATE|Y COORDINATE|   LATITUDE|   LONGITUDE|            LOCATION|
+------------+-------------+-------------------+-------+-----+-----+--------------------+------------+------------+------------+------------+------------+-----------+------------+--------------------+
|Headquarters| Headquarters|3510 S Michigan Ave|Chicago|   IL|60653|http://home.chica...|        null|        null|        null| 1177731.401| 1881697.404|41.83070169|-87.62339535|(41.8307016873, -...|
|          18|   Near North| 1160 N Larrabee St|Chicago|   IL|60610|http://home.chica...|312-742-5870|312-742-5771|312-742-5773| 1172080.029| 1908086.527|41.90324165|-87.64335214|(41.9032416531, -

In [16]:
df_crime_report = spark.read.csv('crime.csv',header=True)
df_crime_report.show(5)

+--------+-----------+--------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|                Date|               Block|IUCR|Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+--------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|10224738|   HY411648|09/05/2015 01:30:...|     043XX S WOOD ST|0486|     BATTERY|DOMESTIC BATTERY ...|           RESIDENCE| false|    true|0924|   

In [23]:
df_crime_report.select("District").show()

+--------+
|District|
+--------+
|     009|
|     015|
|     006|
|     014|
|     015|
|     006|
|     014|
|     010|
|     012|
|     008|
|     008|
|     016|
|     005|
|     002|
|     014|
|     006|
|     015|
|     004|
|     022|
|     016|
+--------+
only showing top 20 rows



In [22]:
df_police_station.select("DISTRICT").show()

+--------------------+
|            DISTRICT|
+--------------------+
|        Headquarters|
|                  18|
|                  19|
|                  20|
|                  22|
|                  24|
|                  25|
|                   1|
|                   2|
|                   3|
|                   4|
|                   5|
|                   6|
|                   7|
|                   8|
|                   9|
|                  10|
|                  11|
|                  12|
|",Chicago,IL,6060...|
+--------------------+
only showing top 20 rows



In [29]:
from pyspark.sql.functions import lpad, col
df_police_station.select(lpad(col("DISTRICT"),3,"0")).show()


+--------------------+
|lpad(DISTRICT, 3, 0)|
+--------------------+
|                 Hea|
|                 018|
|                 019|
|                 020|
|                 022|
|                 024|
|                 025|
|                 001|
|                 002|
|                 003|
|                 004|
|                 005|
|                 006|
|                 007|
|                 008|
|                 009|
|                 010|
|                 011|
|                 012|
|                 ",C|
+--------------------+
only showing top 20 rows



In [32]:
df_police_station = df_police_station.withColumn("formatted_district",lpad(col("DISTRICT"),3,'0'))
df_police_station.show()

+--------------------+-----------------+--------------------+-------+-----+-----+--------------------+------------+------------+------------+------------+------------+-----------+------------+--------------------+------------------+------------------+
|            DISTRICT|    DISTRICT NAME|             ADDRESS|   CITY|STATE|  ZIP|             WEBSITE|       PHONE|         FAX|         TTY|X COORDINATE|Y COORDINATE|   LATITUDE|   LONGITUDE|            LOCATION|formatted district|formatted_district|
+--------------------+-----------------+--------------------+-------+-----+-----+--------------------+------------+------------+------------+------------+------------+-----------+------------+--------------------+------------------+------------------+
|        Headquarters|     Headquarters| 3510 S Michigan Ave|Chicago|   IL|60653|http://home.chica...|        null|        null|        null| 1177731.401| 1881697.404|41.83070169|-87.62339535|(41.8307016873, -...|               Hea|            

In [35]:
df_police_station.columns

['DISTRICT',
 'DISTRICT NAME',
 'ADDRESS',
 'CITY',
 'STATE',
 'ZIP',
 'WEBSITE',
 'PHONE',
 'FAX',
 'TTY',
 'X COORDINATE',
 'Y COORDINATE',
 'LATITUDE',
 'LONGITUDE',
 'LOCATION',
 'formatted district',
 'formatted_district']

In [40]:
df_crime_report.join(df_police_station, df_crime_report.District == df_police_station.formatted_district,'left_outer').drop('DISTRICT',
 'CITY',
 'STATE',
 'ZIP',
 'WEBSITE',
 'PHONE',
 'FAX',
 'TTY',
 'X COORDINATE',
 'Y COORDINATE',
 'LATITUDE',
 'LONGITUDE',
 'LOCATION',
 'formatted district',
 ).show(truncate=False)

+--------+-----------+----------------------+-----------------------+----+------------------+----------------------------------------------+------------------------------+------+--------+----+----+--------------+--------+----+----------------------+--------------+----------------------+------------------+
|ID      |Case Number|Date                  |Block                  |IUCR|Primary Type      |Description                                   |Location Description          |Arrest|Domestic|Beat|Ward|Community Area|FBI Code|Year|Updated On            |DISTRICT NAME |ADDRESS               |formatted_district|
+--------+-----------+----------------------+-----------------------+----+------------------+----------------------------------------------+------------------------------+------+--------+----+----+--------------+--------+----+----------------------+--------------+----------------------+------------------+
|10224738|HY411648   |09/05/2015 01:30:00 PM|043XX S WOOD ST        |0486|BATTE

In [41]:
df_crime_report.select("Primary Type").distinct().show(truncate=False)

+--------------------------------+
|Primary Type                    |
+--------------------------------+
|OFFENSE INVOLVING CHILDREN      |
|CRIMINAL SEXUAL ASSAULT         |
|STALKING                        |
|PUBLIC PEACE VIOLATION          |
|OBSCENITY                       |
|NON-CRIMINAL (SUBJECT SPECIFIED)|
|ARSON                           |
|GAMBLING                        |
|CRIMINAL TRESPASS               |
|ASSAULT                         |
|NON - CRIMINAL                  |
|LIQUOR LAW VIOLATION            |
|MOTOR VEHICLE THEFT             |
|THEFT                           |
|BATTERY                         |
|ROBBERY                         |
|HOMICIDE                        |
|PUBLIC INDECENCY                |
|CRIM SEXUAL ASSAULT             |
|HUMAN TRAFFICKING               |
+--------------------------------+
only showing top 20 rows



In [50]:
df_crime_report.columns

['DISTRICT',
 'DISTRICT NAME',
 'ADDRESS',
 'CITY',
 'STATE',
 'ZIP',
 'WEBSITE',
 'PHONE',
 'FAX',
 'TTY',
 'X COORDINATE',
 'Y COORDINATE',
 'LATITUDE',
 'LONGITUDE',
 'LOCATION',
 'formatted district',
 'formatted_district']

In [66]:
df_crime_report.select("Primary Type").orderBy(col("Primary Type")).distinct().show(36,truncate= False)

+---------------------------------+
|Primary Type                     |
+---------------------------------+
|OFFENSE INVOLVING CHILDREN       |
|CRIMINAL SEXUAL ASSAULT          |
|STALKING                         |
|PUBLIC PEACE VIOLATION           |
|OBSCENITY                        |
|NON-CRIMINAL (SUBJECT SPECIFIED) |
|ARSON                            |
|GAMBLING                         |
|CRIMINAL TRESPASS                |
|ASSAULT                          |
|NON - CRIMINAL                   |
|LIQUOR LAW VIOLATION             |
|MOTOR VEHICLE THEFT              |
|THEFT                            |
|BATTERY                          |
|ROBBERY                          |
|HOMICIDE                         |
|PUBLIC INDECENCY                 |
|CRIM SEXUAL ASSAULT              |
|HUMAN TRAFFICKING                |
|INTIMIDATION                     |
|PROSTITUTION                     |
|DECEPTIVE PRACTICE               |
|CONCEALED CARRY LICENSE VIOLATION|
|SEX OFFENSE                

In [67]:
nc = df_crime_report.filter((col("Primary Type") == "NON - CRIMINAL")|(col("Primary Type") == "NON-CRIMINAL (SUBJECT SPECIFIED)")|(col("Primary Type") == "NON-CRIMINAL"))

In [81]:
nc.groupBy("Description").count().orderBy('count',ascending = False).show()

+--------------------+-----+
|         Description|count|
+--------------------+-----+
|       LOST PASSPORT|  111|
|   FOID - REVOCATION|   76|
|CONCEALED CARRY L...|   14|
|NOTIFICATION OF C...|    9|
|NOTIFICATION OF S...|    8|
|      FOUND PASSPORT|    4|
|GUN OFFENDER NOTI...|    3|
+--------------------+-----+



In [82]:
df_crime_report.show(5)

+--------+-----------+--------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|                Date|               Block|IUCR|Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+--------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|10224738|   HY411648|09/05/2015 01:30:...|     043XX S WOOD ST|0486|     BATTERY|DOMESTIC BATTERY ...|           RESIDENCE| false|    true|0924|   

In [84]:
df_crime_report.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: string (nullable = true)
 |-- Beat: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Ward: string (nullable = true)
 |-- Community Area: string (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: string (nullable = true)
 |-- Y Coordinate: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Location: string (nullable = true)

